In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageFilter
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Layer,Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
positive_path = os.path.join('data', 'positive')
negative_path = os.path.join('data', 'negative')
anchor_path = os.path.join('data', 'anchor')
print(anchor_path)

In [ ]:
POS_PATH

In [ ]:
os.makedirs(positive_path)
os.makedirs(negative_path)
os.makedirs(anchor_path)

In [ ]:
!tar -xf /content/drive/MyDrive/lfw.tgz

In [ ]:
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    old_path = os.path.join('lfw', directory, file)
    new_path = os.path.join(negative_path, file)
    os.replace(old_path,new_path)


In [ ]:
for directory in os.listdir('data/negative'):
  print(directory)

In [ ]:
#Access Image Folders and convert them into tensorflow datasets
anchor = tf.data.Dataset.list_files('/content/drive/MyDrive/data/anchor' + '/*.jpg').take(300)
print("A",anchor)
positive = tf.data.Dataset.list_files('/content/drive/MyDrive/data/positive' + '/*.jpg').take(300)
negative = tf.data.Dataset.list_files('/content/drive/MyDrive/data/negative' + '/*.jpg').take(300)


dir_test = anchor.as_numpy_iterator()

print(dir_test.next())


In [ ]:
#Preprocessing step which includes scaling and resizing of images
def preprocess_image(img_path):
    img = tf.io.read_file(img_path)
    # Load in the image
    img_object = tf.io.decode_jpeg(img)
    print("I",img_object)
    grayscale_imgs = tf.image.rgb_to_grayscale([img_object])
    grayscale_img = grayscale_imgs[0]
    print("J",grayscale_img)

    # Preprocessing steps - resizing the image to be 100x100x3
    grayscale_img = tf.image.resize(grayscale_img, (105,105))

    #print("K",grayscale_img)
    # Scale image to be between 0 and 1
    grayscale_img = grayscale_img / 255.0

    # Return image
    return grayscale_img






In [ ]:
image = preprocess_image('/content/drive/MyDrive/data/positive/02b46c2b-2bd1-11ee-b6d4-5cbaefd5b04c.jpg')
print(image)
plt.imshow(image)


In [ ]:
#Constructing a labelled dataset
negative_samples = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
positive_samples = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
data = positive_samples.concatenate(negative_samples)
print(len(data))


In [ ]:
#Splitting the dataset into training and testing datasets
def preprocess_data(img1,img2,label):
  return(preprocess_image(img1), preprocess_image(img2), label)
#Preprocessing the entire dataset
for i in data:
  print("I",i)
data = data.map(preprocess_data)
#Caching the dataset
data = data.cache()
#Shuffling the elements of the dataset
data = data.shuffle(buffer_size=1024)
samples = data.as_numpy_iterator()
samp = samples.next()
plt.imshow(samp[0])


In [ ]:
print(len(data))

In [ ]:
#Extract training data
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

#Extracting testing data
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
print(type(test_data))

In [ ]:
#Creating the model which involves the creating the embedding layer follwed by creating the distance layer

#Creating the embedding layer
inp = Input(shape=(105,105,1), name='input_image')
c1 = Conv2D(64, (10,10), activation='relu')(inp)
m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
c4 = Conv2D(256, (4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)
embedding_layer = Model(inputs=[inp], outputs=[d1], name='embedding_layer')
embedding_layer.summary()

In [ ]:
#Creating the distance layer
def L1distLayer(input1,input2):
  return tf.math.abs(input1 - input2)

In [ ]:
#Constructing the final Siamese Model
input_img = Input(name='input_img', shape=(105,105))
ver_img = Input(name='verification_img', shape=(105,105))
inp_embedding = embedding_layer(input_img )
ver_embedding = embedding_layer(ver_img)


distances = L1distLayer(inp_embedding, ver_embedding)
final_classifier = Dense(1, activation='sigmoid')(distances)
final_network = Model(inputs=[input_img, ver_img], outputs=final_classifier, name='SiameseNetwork')
final_network.summary()




In [ ]:
final_network.summary()

In [ ]:
#Choose a loss function(Binary Cross Entropy in this case)
loss = tf.losses.BinaryCrossentropy()

#Choose an optimizer(Adam in this case)
optimizer = tf.keras.optimizers.Adam(1e-4)   #Learning rate = 1e-4


In [ ]:
#Create checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=optimizer, siamese_model= final_network)

In [ ]:
#Implementing the training function

def train(batch):

    # Record all of our operations
    with tf.GradientTape() as tape:
        # Get anchor and positive/negative image
        x = batch[:2]
        # Get label
        y = batch[2]

        # Forward pass
        y_cal = final_network(x, training=True)
        # Calculate loss
        b_loss = loss(y, y_cal)
    print(b_loss,type(b_loss))

    # Calculate gradients
    print("L")
    grad = tape.gradient(b_loss, final_network.trainable_variables)
    print("K")

    # Calculate updated weights and apply to siamese model
    optimizer.apply_gradients(zip(grad, final_network.trainable_variables))

    # Return loss
    return b_loss

In [ ]:
#Implementing the training loop
def train_model(data, epochs):
    # Loop through epochs
    for epoch in range(1, epochs+1):
        print('\n Epoch {}/{}'.format(epoch, epochs))
        progress_bar = tf.keras.utils.Progbar(len(data))

        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            train(batch)
            progress_bar.update(idx+1)

        # Save checkpoints
        if epoch % 2 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
#Training the model
train_model(train_data, 15)

In [ ]:
print("a")

In [ ]:
print("B")

In [ ]:
print("N")

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()
y_hat = final_network.predict([test_input, test_val])
y_hat

In [ ]:
from tensorflow.keras.metrics import Precision, Recall
m = Recall()

# Calculating the recall value
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
final_network.save('siamesemodel.h5')

In [ ]:
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/siameseNetwork.h5',
                                   custom_objects={'L1Dist': L1distLayer, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})
siamese_model.summary()

In [ ]:
#Verification using Real Time Dataset
input_img = preprocess_image('/content/drive/MyDrive/IMG_20230212_160419.jpg')
anchor_img = preprocess_image('')
result = siamese_model.predict(list(np.expand_dims([input_img, anchor_img], axis=1)))
print(result)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')